In [1]:
# Import Libraries

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

# Get Dataset 

In [3]:
dataset_path = '/kaggle/input/cifar-10/sampleSubmission.csv'
data = pd.read_csv(dataset_path)
print(data.head())

# You need to provide the actual path to the dataset files
dataset_directory = '/kaggle/input/cifar-10/dataset/'  # Update this path

   id label
0   1   cat
1   2   cat
2   3   cat
3   4   cat
4   5   cat


#  Split Data

In [4]:
# Split the data into features (X) and labels (Y)
X = data['id']
Y = data['label']

# Split data for trainging and testing purpose
train_images, test_images, train_labels, test_labels = train_test_split(X, Y, test_size=0.2, random_state=42)


# Model Architecture

In [5]:
# Define the CNN model
model = models.Sequential()

# Convolutional and Pooling layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten layer
model.add(layers.Flatten())

# Fully connected layers
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))  # Assuming 10 classes

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 128)        0

In [6]:
# Create a dictionary to map class names to integer labels
class_to_label = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
                  'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}              # This dictionary class_to_label is used to map class names (such as 'airplane', 'automobile', etc.) to their corresponding integer labels. This mapping is essential for training machine learning models, as models require labels in numeric format.

# Convert label names to integer labels
train_labels_int = [class_to_label[label] for label in train_labels]
test_labels_int = [class_to_label[label] for label in test_labels]

# Convert labels to one-hot encoded vectors
num_classes = 10  # Replace with the actual number of classes
train_labels_encoded = tf.keras.utils.to_categorical(train_labels_int, num_classes=num_classes)
test_labels_encoded = tf.keras.utils.to_categorical(test_labels_int, num_classes=num_classes)

# Preprocess the Data

In [7]:
def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, (32, 32))  # Resize to your desired dimensions
    return image, label

# Create Datasets

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels_encoded))
train_dataset = train_dataset.map(preprocess_image)  # Apply preprocess_image function
train_dataset = train_dataset.shuffle(buffer_size=len(train_images)).batch(32)

# Create a dataset from tensors of test_images and test_labels
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels_encoded))

# Apply the preprocess_image function to each sample in the test dataset
test_dataset = test_dataset.map(preprocess_image)

# Create batches of size 32 for the test dataset
test_dataset = test_dataset.batch(32)

# Train the model

In [9]:
# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_dataset)

# # Print the test loss and accuracy
# # print(f"Test Loss: {loss:.4f}")
# # print(f"Test Accuracy: {accuracy:.4f}")